# Imports

In [1]:
import pandas as pd

from khan_helpers import Experiment
from khan_helpers.functions import get_top_words, show_source

Experiment & Participant classes, helper functions, and variables used across multiple notebooks can be found in `/opt/conda/lib/python3.9/site-packages/khan_helpers`, or on GitHub, [here](https://github.com/contextlab/efficient-learning-khan/tree/master/code/khan_helpers).<br />You can also view source code directly from the notebook with:<br /><pre>    from khan_helpers.functions import show_source<br />    show_source(foo)</pre>

# Show function defined in `khan_helpers`

In [2]:
show_source(get_top_words)

In [3]:
exp = Experiment()
questions = exp.questions

# Table S1

In [4]:
questions.index.name = 'ID'
questions.columns = ['Question set', 
                     'Question', 
                     'Correct response', 
                     'Alternative 1', 
                     'Alternative 2', 
                     'Alternative 3']
questions['Question set'] = questions['Question set'].replace({1: 'FFF', 
                                                               2: 'BoS', 
                                                               0: 'GPK'})

col_format = 'r|p{0.375in}|p{1.275in}|p{0.75in}|p{0.75in}|p{0.75in}|p{0.75in}'
questions.head()

,Question set,Question,Correct response,Alternative 1,Alternative 2,Alternative 3
ID,,,,,,
1,FFF,Why is the gravitational attraction between yo...,Neither you nor your computer has enough mass ...,You and your computer are too close for the gr...,Humans are too small to detect the force of gr...,The gravitational attraction between you and y...
2,FFF,Which of the following is an example of the We...,A neutron in a radioactive Cesium atom is conv...,Light from the sun collides with a satellite o...,Two protons bound together in a Helium nucleus...,A distant galaxy exerts a small but detectable...
3,FFF,Roughly how many times stronger is the Weak In...,"10,000,000,000,000,000,000,000,000",10,"1,000,000",The Weak Interaction is less strong than gravity
4,FFF,Why don't you and your computer experience any...,The weak interaction only acts over extremely ...,The weak interaction between you and your comp...,You and your computer have no net charge,Neither you nor your computer has enough mass ...
5,FFF,Which of the following is a difference between...,Gravity is only ever attractive while the elec...,Gravity is a much more powerful force than ele...,Gravity can only act over large distances whil...,The electromagnetic force can only act over sm...


In [5]:
with pd.option_context("max_colwidth", 300):
    latex_code = questions.to_latex(longtable=True, column_format=col_format)
    print(latex_code.replace(r'\\', r'\\\hline'))

\begin{longtable}{r|p{0.375in}|p{1.275in}|p{0.75in}|p{0.75in}|p{0.75in}|p{0.75in}}
\toprule
{} & Question set &                                                                                                                                                                                                                                                                         Question &                                                                                                                      Correct response &                                                                                         Alternative 1 &                                                                                                              Alternative 2 &                                                                                                                     Alternative 3 \\\hline
ID &              &                                                                                          

# Table S2

In [6]:
top_words = pd.DataFrame(get_top_words(exp.fit_cv, exp.fit_lda)).T
top_words.columns = range(1, len(top_words.columns) + 1)
top_words.index = range(1, len(top_words.index) + 1)
top_words.columns.name = 'Topic'
top_words.head()

Topic,1,2,3,4,5,6,7,8,9,10
1,star,helium,main,mass,atomic,sequence,get,energy,fuse,hydrogen
2,charge,force,mass,gravity,strong,attract,large,strength,distance,electromagnetic
3,huge,force,electromagnetic,macro,way,scale,concentration,apply,kind,charge
4,atom,dense,go,hydrogen,slow,get,huge,condense,mass,would
5,fusion,get,threshold,core,occur,mass,something,start,several,jupiter


In [7]:
print(top_words.to_latex().replace(r'\\', r'\\\hline'))

\begin{tabular}{lllllllllll}
\toprule
Topic &       1  &        2  &               3  &           4  &       5  &        6  &             7  &           8  &        9  &               10 \\\hline
\midrule
1  &     star &    helium &             main &         mass &   atomic &  sequence &            get &       energy &      fuse &         hydrogen \\\hline
2  &   charge &     force &             mass &      gravity &   strong &   attract &          large &     strength &  distance &  electromagnetic \\\hline
3  &     huge &     force &  electromagnetic &        macro &      way &     scale &  concentration &        apply &      kind &           charge \\\hline
4  &     atom &     dense &               go &     hydrogen &     slow &       get &           huge &     condense &      mass &            would \\\hline
5  &   fusion &       get &        threshold &         core &    occur &      mass &      something &        start &   several &          jupiter \\\hline
6  &   enough &  ign